In [1]:
import pandas as pd
import numpy as np
from database_manager import DatabaseManager

db = DatabaseManager()
# 按时间顺序拉全量比赛
cursor = db.connection.cursor()
cursor.execute("""
    SELECT match_id, season, match_date, home_team_id, away_team_id,
           home_score, away_score, result
    FROM matches
    ORDER BY match_date;
""")
df = pd.DataFrame(cursor.fetchall())
cursor.close()
print(f'共 {df.shape[0]} 场比赛')

✅ 数据库连接成功
共 174 场比赛


In [2]:
# 只留数字列，方便计算
df['home_goals'] = df['home_score'].astype(int)
df['away_goals'] = df['away_score'].astype(int)
df['home_win']   = (df['result'] == 'H').astype(int)

# ----- 特征 1：主队近 3 场进球数 -----
df['home_last3_goals'] = (
    df.groupby('home_team_id')['home_goals']
      .rolling(3, min_periods=1).sum()
      .shift(1)          # 不包含本场
      .reset_index(level=0, drop=True)
)

# ----- 特征 2：客队近 3 场失球数 -----
df['away_last3_concede'] = (
    df.groupby('away_team_id')['home_goals']
      .rolling(3, min_periods=1).sum()
      .shift(1)
      .reset_index(level=0, drop=True)
)

# ----- 特征 3：主队是否连败（近 2 场全输） -----
df['home_last2_loss'] = (
    df.groupby('home_team_id')['home_win']
      .rolling(2, min_periods=1).sum()
      .shift(1)
      .reset_index(level=0, drop=True)
) == 0

df[['season', 'home_team_id', 'home_last3_goals', 'away_last3_concede', 'home_last2_loss']].head()

,season,home_team_id,home_last3_goals,away_last3_concede,home_last2_loss
0,2019,31,2.0,6.0,True
1,2019,29,12.0,1.0,True
2,2019,26,2.0,3.0,True
3,2019,7,5.0,4.0,True
4,2019,15,5.0,5.0,True


In [3]:
# 只留建模需要列
feat_df = df[['season', 'home_team_id', 'away_team_id',
              'home_last3_goals', 'away_last3_concede', 'home_last2_loss',
              'result']].dropna()

feat_df.to_csv(r'E:\miniconda\ChineseSuperLeague\data\features.csv', index=False)
print(f'特征表保存完成，共 {feat_df.shape[0]} 行')

特征表保存完成，共 172 行
